In [1]:
# Import data set
import requests

url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)

with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
print('Length of dataset in characters: ', len(text))

chars = sorted(list(set(text))) # Here are all the unique characters that occur in this text
vocab_size = len(chars)
print(''.join(chars))
print('Vocab size', vocab_size)

Length of dataset in characters:  1115394

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Vocab size 65


In [7]:
# Lookup tables (tokenizer)
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]             # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l])    # decoder: take a list of integers, output a string
print(encode("Hello world"))
print(decode(encode("Hello world")))

[20, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
Hello world


In [8]:
# Encode the entire text dataset and store it into a torch.tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [9]:
# Split the data bewteen the training set and the validation set
# Helps us understand to what extent the model is overfitting
# We are using 90% as training data and 10% as validation data (if we were to use 100% of as validation data oit would produce the same input)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [11]:
# The block size will give us context on what the next probable character is
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [13]:
x = train_data[:block_size]
y = train_data[1:block_size +1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'Input: {context}, Output: {target}')

Input: tensor([18]), Output: 47
Input: tensor([18, 47]), Output: 56
Input: tensor([18, 47, 56]), Output: 57
Input: tensor([18, 47, 56, 57]), Output: 58
Input: tensor([18, 47, 56, 57, 58]), Output: 1
Input: tensor([18, 47, 56, 57, 58,  1]), Output: 15
Input: tensor([18, 47, 56, 57, 58,  1, 15]), Output: 47
Input: tensor([18, 47, 56, 57, 58,  1, 15, 47]), Output: 58
